In [25]:
# ============================================================
# STEP 0 — Imports + determinisme
# ============================================================

import numpy as np
import random
import joblib

from scipy.stats import skew, kurtosis

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

np.random.seed(42)
random.seed(42)

# ============================================================
# STEP 1 — Load data
# ============================================================

X = np.load("X_windows.npy")     # (N, 200, 3)
y = np.load("y_labels.npy")

# ============================================================
# STEP 2 — Goertzel band energy
# ============================================================

def goertzel_band_energy(x, fs, f_low, f_high):
    x = np.asarray(x)
    n = len(x)
    energy = 0.0

    for f in range(int(f_low), int(f_high) + 1):
        k = int(0.5 + (n * f) / fs)
        w = 2.0 * np.pi * k / n
        coeff = 2.0 * np.cos(w)

        s_prev = 0.0
        s_prev2 = 0.0

        for sample in x:
            s = sample + coeff * s_prev - s_prev2
            s_prev2 = s_prev
            s_prev = s

        power = s_prev2**2 + s_prev**2 - coeff * s_prev * s_prev2
        energy += power

    return energy

# ============================================================
# STEP 3 — Statistik-features (samme som før)
# ============================================================

def extract_stats(v):
    p10, p25, p75, p90 = np.percentile(v, [10, 25, 75, 90])
    return [
        np.mean(v),
        np.std(v),
        np.var(v),
        np.min(v),
        np.max(v),
        np.median(v),

        p10, p25, p75, p90,

        np.sqrt(np.mean(v**2)),            # RMS
        np.sum(v**2),                      # Energy
        np.max(v) - np.min(v),             # Peak-to-peak
        np.mean(np.abs(v - np.mean(v))),   # MAD
        skew(v),
        kurtosis(v),
    ]

# ============================================================
# STEP 4 — Feature extraction (GOERTZEL)
# ============================================================

def extract_features(window, fs=100):
    x = window[:, 0]
    y_ = window[:, 1]
    z = window[:, 2]
    mag = np.sqrt(x**2 + y_**2 + z**2)

    features = []

    # Statistik + Goertzel pr. akse
    for v in [x, y_, z]:
        features += extract_stats(v)
        features += [
            0.0,  # dom_freq (unused)
            0.0,  # centroid (unused)
            0.0,  # spec_entropy (unused)
            goertzel_band_energy(v, fs, 0, 5),
            goertzel_band_energy(v, fs, 5, 15),
            goertzel_band_energy(v, fs, 15, 40),
        ]

    # Magnitude
    features += extract_stats(mag)
    features += [
        0.0,
        0.0,
        0.0,
        goertzel_band_energy(mag, fs, 0, 5),
        goertzel_band_energy(mag, fs, 5, 15),
        goertzel_band_energy(mag, fs, 15, 40),
    ]

    # Korrelationsfeatures
    features += [
        np.corrcoef(x, y_)[0, 1],
        np.corrcoef(x, z)[0, 1],
        np.corrcoef(y_, z)[0, 1],
    ]

    return features

# ============================================================
# STEP 5 — Feature extraction
# ============================================================

print("\nExtracting GOERTZEL features…")

F = np.array([extract_features(win) for win in X])
print("Feature matrix shape:", F.shape)

# ============================================================
# STEP 6 — Feature names (SAMME rækkefølge!)
# ============================================================

def get_feature_names():
    names = []

    axes = ["x", "y", "z"]
    stats_names = [
        "mean", "std", "var", "min", "max", "median",
        "p10", "p25", "p75", "p90",
        "rms", "energy", "ptp", "mad", "skew", "kurt",
    ]

    freq_names = [
        "dom_freq", "centroid", "spec_entropy",
        "band0_5", "band5_15", "band15_40"
    ]

    for a in axes:
        for s in stats_names:
            names.append(f"{a}_{s}")
        for f in freq_names:
            names.append(f"{a}_{f}")

    for s in stats_names:
        names.append(f"mag_{s}")
    for f in freq_names:
        names.append(f"mag_{f}")

    names += ["corr_xy", "corr_xz", "corr_yz"]
    return names

feature_names = get_feature_names()

# ============================================================
# STEP 7 — LÅS TOP-10 FEATURES
# ============================================================

TOP10_FEATURES = [
    "y_band15_40",
    "y_band5_15",
    "mag_band15_40",
    "x_band15_40",
    "z_band15_40",
    "y_kurt",
    "y_rms",
    "y_energy",
    "y_max",
    "y_p10",
]

top10_indices = [feature_names.index(f) for f in TOP10_FEATURES]
F10 = F[:, top10_indices]

# ============================================================
# STEP 8 — Labels
# ============================================================

le = LabelEncoder()
y_enc = le.fit_transform(y)

# ============================================================
# STEP 9 — Train/test split
# ============================================================

X_train, X_test, y_train, y_test = train_test_split(
    F10, y_enc,
    test_size=0.2,
    random_state=42,
    stratify=y_enc
)

# ============================================================
# STEP 10 — Random Forest
# ============================================================

rf10 = RandomForestClassifier(
    n_estimators=400,
    max_depth=20,
    min_samples_leaf=2,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

rf10.fit(X_train, y_train)

# ============================================================
# STEP 11 — Evaluering
# ============================================================

y_pred = rf10.predict(X_test)
print("Goertzel RF accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))

# ============================================================
# STEP 12 — FRYS MODELLEN
# ============================================================

joblib.dump(
    {
        "model": rf10,
        "feature_names": TOP10_FEATURES,
        "label_classes": le.classes_,
        "frequency_method": "goertzel"
    },
    "rf10_goertzel_frozen.joblib"
)

print("✔ Goertzel-model frosset og gemt som rf10_goertzel_frozen.joblib")



Extracting GOERTZEL features…
Feature matrix shape: (757, 91)
Goertzel RF accuracy: 0.868421052631579
              precision    recall  f1-score   support

      normal       0.83      0.88      0.86        68
   udrykning       0.90      0.86      0.88        84

    accuracy                           0.87       152
   macro avg       0.87      0.87      0.87       152
weighted avg       0.87      0.87      0.87       152

✔ Goertzel-model frosset og gemt som rf10_goertzel_frozen.joblib


In [26]:
y1 = rf10.predict(X_test)
y2 = rf10.predict(X_test)

assert np.all(y1 == y2)
print("✔ Model deterministisk og frosset")

✔ Model deterministisk og frosset


In [15]:
# ================================================
# Python ↔ C feature-match sanity check
# ================================================

# Vælg én window
idx = 0   # du kan ændre index hvis du vil
win = X[idx]

# Udregn ALLE features
features = extract_features(win)

# Lås TOP10 i korrekt rækkefølge
TOP10_FEATURES = [
    "y_band15_40",
    "y_band5_15",
    "mag_band15_40",
    "x_band15_40",
    "z_band15_40",
    "y_kurt",
    "y_rms",
    "y_energy",
    "y_max",
    "y_p10",
]

top10_indices = [feature_names.index(f) for f in TOP10_FEATURES]

f10 = [features[i] for i in top10_indices]

print("TOP10 features (Python):")
for i, v in enumerate(f10):
    print(f"{i}: {v:.6f}")

print("\nPython prediction:", rf10.predict([f10])[0])

TOP10 features (Python):
0: 0.000891
1: 0.003351
2: 0.000826
3: 0.000788
4: 0.001793
5: 2.194321
6: 0.199042
7: 7.923541
8: 0.050000
9: -0.270000

Python prediction: 0


In [16]:
import numpy as np

def goertzel_band_energy(x, fs, f_low, f_high):
    """
    Approximer båndenergi vha. Goertzel.
    Summerer energien for frekvenser i [f_low, f_high].
    """
    n = len(x)
    freqs = np.arange(int(f_low), int(f_high) + 1)

    energy = 0.0
    for f in freqs:
        k = int(0.5 + (n * f) / fs)
        w = 2 * np.pi * k / n
        coeff = 2 * np.cos(w)

        s_prev = 0.0
        s_prev2 = 0.0

        for sample in x:
            s = sample + coeff * s_prev - s_prev2
            s_prev2 = s_prev
            s_prev = s

        power = s_prev2**2 + s_prev**2 - coeff * s_prev * s_prev2
        energy += power

    return energy

In [17]:
from scipy.signal import welch

fs = 100
win = X[0]

x = win[:,0]
y = win[:,1]
z = win[:,2]
mag = np.sqrt(x*x + y*y + z*z)

def welch_band(v, f1, f2):
    freqs, psd = welch(v, fs=fs, nperseg=len(v))
    return np.sum(psd[(freqs >= f1) & (freqs < f2)])

print("=== Y axis ===")
print("Welch 5–15 :", welch_band(y, 5, 15))
print("Goertzel 5–15 :", goertzel_band_energy(y, fs, 5, 15))

print("Welch 15–40 :", welch_band(y, 15, 40))
print("Goertzel 15–40 :", goertzel_band_energy(y, fs, 15, 40))

=== Y axis ===
Welch 5–15 : 0.003351451
Goertzel 5–15 : 33.11220983342233
Welch 15–40 : 0.0008913937
Goertzel 15–40 : 17.35157280578659


In [18]:
from tqdm import trange
from scipy.stats import pearsonr

N = 200  # antal windows at teste

welch_vals = []
goertzel_vals = []

for i in trange(N):
    win = X[i]
    y = win[:,1]

    w = welch_band(y, 15, 40)
    g = goertzel_band_energy(y, fs, 15, 40)

    welch_vals.append(w)
    goertzel_vals.append(g)

welch_vals = np.array(welch_vals)
goertzel_vals = np.array(goertzel_vals)

corr, _ = pearsonr(welch_vals, goertzel_vals)
print("Pearson correlation:", corr)

100%|██████████| 200/200 [00:00<00:00, 471.59it/s]

Pearson correlation: 0.8216607949460979


In [19]:
# Sammenlign prediction med Welch vs Goertzel features
idx = 0
win = X[idx]

features = extract_features(win)
top10_indices = [feature_names.index(f) for f in TOP10_FEATURES]

# Erstat Welch-features med Goertzel i f10
f10_goertzel = []

for name in TOP10_FEATURES:
    if name == "y_band15_40":
        f10_goertzel.append(goertzel_band_energy(y, fs, 15, 40))
    elif name == "y_band5_15":
        f10_goertzel.append(goertzel_band_energy(y, fs, 5, 15))
    else:
        f10_goertzel.append(features[feature_names.index(name)])

print("Python (Welch) prediction:", rf10.predict([[features[i] for i in top10_indices]])[0])
print("Python (Goertzel) prediction:", rf10.predict([f10_goertzel])[0])

Python (Welch) prediction: 0
Python (Goertzel) prediction: 0


In [22]:
same = 0
diff = 0

for i in range(200):
    win = X[i]
    x = win[:,0]
    y = win[:,1]
    z = win[:,2]
    mag = np.sqrt(x*x + y*y + z*z)

    features = extract_features(win)
    f10_welch = [features[feature_names.index(f)] for f in TOP10_FEATURES]

    f10_goertzel = []
    for name in TOP10_FEATURES:
        if name == "y_band15_40":
            f10_goertzel.append(goertzel_band_energy(y, 100, 15, 40))
        elif name == "y_band5_15":
            f10_goertzel.append(goertzel_band_energy(y, 100, 5, 15))
        elif name == "mag_band15_40":
            f10_goertzel.append(goertzel_band_energy(mag, 100, 15, 40))
        elif name == "x_band15_40":
            f10_goertzel.append(goertzel_band_energy(x, 100, 15, 40))
        elif name == "z_band15_40":
            f10_goertzel.append(goertzel_band_energy(z, 100, 15, 40))
        else:
            f10_goertzel.append(features[feature_names.index(name)])

    if rf10.predict([f10_welch])[0] == rf10.predict([f10_goertzel])[0]:
        same += 1
    else:
        diff += 1

print("Same predictions:", same)
print("Different predictions:", diff)
print("Agreement:", same / (same + diff))

Same predictions: 178
Different predictions: 22
Agreement: 0.89


In [27]:
# Vælg et rigtigt window
idx = 0
win = X[idx]

# Udregn Goertzel-features
features = extract_features(win)

# Lås TOP10 i korrekt rækkefølge
f10_py = [features[feature_names.index(f)] for f in TOP10_FEATURES]

print("Python Goertzel TOP10:")
for i, v in enumerate(f10_py):
    print(f"{i}: {v:.6f}")

Python Goertzel TOP10:
0: 17.351573
1: 33.112210
2: 8.330667
3: 7.605834
4: 14.859246
5: 2.194321
6: 0.199042
7: 7.923541
8: 0.050000
9: -0.270000


In [28]:
# Print window som C-array (float)
for i in range(200):
    print(f"{{{win[i,0]:.6f}f, {win[i,1]:.6f}f, {win[i,2]:.6f}f}},")

{0.880000f, -0.370000f, 0.180000f},
{0.830000f, -0.370000f, 0.230000f},
{0.870000f, -0.280000f, 0.030000f},
{0.880000f, -0.390000f, 0.060000f},
{0.940000f, -0.490000f, 0.110000f},
{0.790000f, -0.480000f, 0.130000f},
{0.860000f, -0.350000f, 0.210000f},
{0.910000f, -0.250000f, 0.210000f},
{0.980000f, -0.200000f, 0.320000f},
{1.010000f, -0.130000f, 0.130000f},
{0.930000f, -0.190000f, 0.200000f},
{0.920000f, -0.220000f, 0.190000f},
{0.870000f, -0.290000f, 0.270000f},
{1.020000f, -0.230000f, 0.020000f},
{0.960000f, -0.320000f, 0.060000f},
{0.930000f, -0.410000f, 0.110000f},
{0.810000f, -0.380000f, 0.240000f},
{0.820000f, -0.230000f, 0.290000f},
{0.890000f, -0.110000f, 0.200000f},
{1.020000f, -0.110000f, 0.050000f},
{1.010000f, -0.300000f, 0.070000f},
{0.950000f, -0.360000f, 0.210000f},
{0.920000f, -0.250000f, 0.210000f},
{0.890000f, -0.140000f, 0.180000f},
{0.950000f, -0.140000f, 0.120000f},
{0.940000f, -0.230000f, 0.160000f},
{0.990000f, -0.330000f, 0.150000f},
{0.970000f, -0.270000f, 0.10